In [1]:
# Cài đặt các thư viện cốt lõi cho RAG và LLM
!pip install -q \
    torch \
    transformers \
    accelerate \
    bitsandbytes \
    llama-index \
    llama-index-readers-file \
    llama-index-llms-huggingface \
    llama-index-embeddings-huggingface \
    # Thư viện cho Jupyter/Colab để hiển thị tiến trình


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

In [2]:
# Cài đặt các thư viện cốt lõi cho RAG và LLM
!pip install -q \
    torch \
    transformers \
    accelerate \
    bitsandbytes \
    llama-index \
    llama-index-readers-file \
    llama-index-llms-huggingface \
    llama-index-embeddings-huggingface \
    # Thư viện cho Jupyter/Colab để hiển thị tiến trình


In [3]:
!pip install pypdf
!pip install -q tqdm

In [4]:
!hf auth login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Y
Token is valid (permission: read).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pus

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import os, shutil, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from llama_index.core import (
    Settings,
    VectorStoreIndex,
    load_index_from_storage,
    StorageContext,
)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.ingestion.cache import IngestionCache
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import SummaryExtractor, TitleExtractor
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM


In [16]:
# --- ĐỊNH NGHĨA CÁC BIẾN TOÀN CỤC ---
DOCX_PATH = "/content/drive/MyDrive/DACNTT/DMS-5.pdf"
CACHE_FILE = "/content/drive/MyDrive/DACNTT/data/cache/pipeline_cache.json"
INDEX_STORAGE = "/content/drive/MyDrive/DACNTT/data/index_store"

os.makedirs(os.path.dirname(CACHE_FILE), exist_ok=True)
os.makedirs(INDEX_STORAGE, exist_ok=True)
print(f"✅ (Bước 1) Các đường dẫn đã được định nghĩa. DOCX_PATH: {DOCX_PATH}")
# --- KẾT THÚC ĐỊNH NGHĨA BIẾN ---



✅ (Bước 1) Các đường dẫn đã được định nghĩa. DOCX_PATH: /content/drive/MyDrive/DACNTT/DMS-5.pdf


In [17]:

def setup_llm_and_embedding():
    global embed_model, llm

    # ---- Embedding model ----
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    print(f"⏳ (Bước 1) Đang tải Embedding model: {EMBEDDING_MODEL}")
    embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL)
    Settings.embed_model = embed_model
    print("✅ (Bước 1) Embedding Model sẵn sàng.\n")

    # ---- LLM model (Llama 3.1 - 8B 4bit) ----
    LLM_MODEL_NAME = "/content/drive/MyDrive/llama3-8b-mental-health-merged"
    print(f"⏳ (Bước 1) Đang tải LLM 4-bit: {LLM_MODEL_NAME}")

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # --- SỬA LỖI 1 (Ổ khóa & Chìa khóa) ---
    print("ℹ️ (Bước 1) Đang gán 'chat_template'...")
    tokenizer.chat_template = (
        "<|begin_of_text|>"
        "{% for message in messages %}"
            "{% if message['role'] == 'user' %}"
                "<|start_header_id|>user<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>" # FIXED: Removed the extra dot
            "{% elif message['role'] == 'system' %}"
                "<|start_header_id|>system<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>"
            "{% elif message['role'] == 'assistant' %}"
                "<|start_header_id|>assistant<|end_header_id|>\n\n{{ message['content'] }}<|eot_id|>"
            "{% endif %}"
        "{% endfor %}"
    )
    print("✅ (Bước 1) Đã gán 'chat_template' vào 'tokenizer' thành công.")
    # --- KẾT THÚC SỬA LỖI 1 ---

    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # --- SỬA LỖI 2 (Lỗi Lặp - Looping Error) ---
    # Lấy ID của các token dừng (stop tokens)
    eos_token_id = tokenizer.eos_token_id
    # Llama 3 dùng <|eot_id|> (End of Turn) làm tín hiệu dừng
    eot_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")

    print(f"ℹ️ (Bước 1) Đang gán 'stopping_ids' (ID: {eos_token_id} và {eot_token_id})...")
    # --- KẾT THÚC SỬA LỖI 2 ---

    llm = HuggingFaceLLM(
        model=model,
        tokenizer=tokenizer, # <-- Đưa tokenizer đã sửa lỗi 1 vào đây
        context_window=8192,
        max_new_tokens=512,
        generate_kwargs={
            "temperature": 0.3,
            "pad_token_id": eos_token_id
        },
        # --- THÊM THAM SỐ SỬA LỖI 2 VÀO ĐÂY ---
        stopping_ids=[eos_token_id, eot_token_id]
    )

    Settings.llm = llm
    print("✅ (Bước 1) Llama 3.1-8B 4bit đã sẵn sàng (Đã sửa lỗi lặp).\n")

# Chạy setup ngay lập tức để khởi tạo embed_model và llm
setup_llm_and_embedding()

⏳ (Bước 1) Đang tải Embedding model: sentence-transformers/all-MiniLM-L6-v2
✅ (Bước 1) Embedding Model sẵn sàng.

⏳ (Bước 1) Đang tải LLM 4-bit: /content/drive/MyDrive/llama3-8b-mental-health-merged
ℹ️ (Bước 1) Đang gán 'chat_template'...
✅ (Bước 1) Đã gán 'chat_template' vào 'tokenizer' thành công.


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ℹ️ (Bước 1) Đang gán 'stopping_ids' (ID: 128009 và 128009)...
✅ (Bước 1) Llama 3.1-8B 4bit đã sẵn sàng (Đã sửa lỗi lặp).



In [18]:
def ingest_documents():
    print("📂 (Bước 2) Đọc tài liệu ...")

    if not os.path.exists(DOCX_PATH):
        print(f"❌ LỖI: File không tồn tại tại: {DOCX_PATH}")
        return []

    documents = SimpleDirectoryReader(input_files=[DOCX_PATH], filename_as_id=True).load_data()
    print(f"✅ (Bước 2) Đã tải {len(documents)} tài liệu.")

    if not documents:
        print("⚠️ (Bước 2) Không có tài liệu.")
        return []

    # ---- DÙNG CACHE (RẤT QUAN TRỌNG) ----
    try:
        cached_hashes = IngestionCache.from_persist_path(CACHE_FILE)
        print("🧠 (Bước 2) Dùng lại cache đã lưu.\n")
    except:
        cached_hashes = None
        print("ℹ️ (Bước 2) Không có cache, chạy mới.\n")
    # ---

    # --- PIPELINE NÂNG CAO (CHẬM) ---
    print("🚀 (Bước 2) Đang sử dụng pipeline NÂNG CAO với SummaryExtractor (CHẬM 5-8 TIẾNG)...")
    pipeline = IngestionPipeline(
        transformations=[
            TokenTextSplitter(chunk_size=512, chunk_overlap=20),


            #SummaryExtractor(summaries=["self"], prompt_template=CUSTOM_SUMMARY_EXTRACT_TEMPLATE),

            embed_model # Embed model phải chạy sau extractor
        ],
        cache=cached_hashes, # RẤT QUAN TRỌNG ĐỂ DÙNG CACHE
    )
    # --- KẾT THÚC PIPELINE ---

    nodes = pipeline.run(documents=documents)
    print(f"✅ (Bước 2) Đã tạo {len(nodes)} nodes (và tóm tắt) + tính embedding.")

    # Lưu cache sau khi chạy
    pipeline.cache.persist(CACHE_FILE)
    print(f"✅ (Bước 2) Đã lưu cache vào {CACHE_FILE}")
    return nodes

In [19]:
# =================================================================
# HÀM BUILD INDEX
# =================================================================
def build_indexes(nodes):
    print("⏳ Đang kiểm tra/tạo Index...")
    print(f"⚠️ Không tìm thấy Index. Tiến hành tạo Index mới.")

    if not nodes:
        raise ValueError("Không có nodes đầu vào để tạo Index.")

    storage_context = StorageContext.from_defaults()
    vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
    vector_index.set_index_id("vector")

    print(f"⏳ Đang lưu trữ Index mới vào thư mục: {INDEX_STORAGE}...")
    storage_context.persist(persist_dir=INDEX_STORAGE)
    print("✅ Index mới đã được TẠO và LƯU TRỮ thành công.")

    return vector_index


In [20]:
# HÀM MAIN (CHẠY NGAY)
# =================================================================
def main():
    print("\n" + "="*50)
    print("🔹 BẮT ĐẦU CHẠY QUY TRÌNH BUILD DATA...")
    print("="*50)

    try:
        # 1️⃣ Ingest
        nodes = ingest_documents()
        print(f"✅ Ingestion hoàn tất. Đã tạo {len(nodes)} nodes.")

        # 2️⃣ Build index
        if nodes:
            index = build_indexes(nodes)
            print("✅ Hoàn tất xây dựng VectorStoreIndex.")
            return index
        else:
            print("⚠️ Không có nodes nào được tạo, bỏ qua bước Indexing.")
            return None
    except Exception as e:
        print(f"❌ LỖI NGHIÊM TRỌNG TRONG HÀM MAIN: {e}")
        print("👉 Hãy kiểm tra lại các bước 'setup' và đường dẫn file.")
        return None

# --- CHẠY MAIN NGAY LẬP TỨC ---
if 'embed_model' in globals(): # Chỉ chạy nếu Bước 1 đã thành công
    vector_index = main()
else:
    print("❌ BỎ QUA Bước 2: Vui lòng chạy Bước 1 (Setup) trước.")



🔹 BẮT ĐẦU CHẠY QUY TRÌNH BUILD DATA...
📂 (Bước 2) Đọc tài liệu ...
✅ (Bước 2) Đã tải 106 tài liệu.
🧠 (Bước 2) Dùng lại cache đã lưu.

🚀 (Bước 2) Đang sử dụng pipeline NÂNG CAO với SummaryExtractor (CHẬM 5-8 TIẾNG)...
✅ (Bước 2) Đã tạo 293 nodes (và tóm tắt) + tính embedding.
✅ (Bước 2) Đã lưu cache vào /content/drive/MyDrive/DACNTT/data/cache/pipeline_cache.json
✅ Ingestion hoàn tất. Đã tạo 293 nodes.
⏳ Đang kiểm tra/tạo Index...
⚠️ Không tìm thấy Index. Tiến hành tạo Index mới.
⏳ Đang lưu trữ Index mới vào thư mục: /content/drive/MyDrive/DACNTT/data/index_store...
✅ Index mới đã được TẠO và LƯU TRỮ thành công.
✅ Hoàn tất xây dựng VectorStoreIndex.


In [21]:

!pip install -q streamlit pyngrok

In [ ]:
import traceback
import torch
from llama_index.core import Settings, VectorStoreIndex, load_index_from_storage, StorageContext
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.prompts import PromptTemplate
from llama_index.core.llms import ChatMessage
from sentence_transformers import CrossEncoder

# --- Giả sử các biến llm, vector_index đã được load từ các ô trước ---
try:
    from __main__ import llm, vector_index
    if llm is None or vector_index is None:
        raise ImportError
    print("✅ LLM và Vector Index đã sẵn sàng.")
except ImportError:
    print("❌ LỖI: Không tìm thấy 'llm' hoặc 'vector_index'. Vui lòng chạy các ô setup trước.")
    # Dừng ở đây nếu chưa setup
    raise SystemExit("Vui lòng chạy các ô setup LLM và Index trước.")


# --- SỬA LỖI 1: DÙNG SYSTEM PROMPT ĐỂ FIX LỖI SAI VAI ---
# Đây là hướng dẫn cố định cho bot, giúp nó không bị nhầm vai
SYSTEM_PROMPT = """\
Bạn là một **chuyên gia tâm lý AI** (Trợ lý) được phát triển bởi **AI VIETNAM**.
Nhiệm vụ của bạn là trò chuyện, theo dõi và tư vấn cho người dùng (User) về sức khỏe tâm thần.
Luôn luôn hành động với tư cách là Trợ lý, thể hiện sự đồng cảm và chuyên nghiệp.

QUY TẮC RAG:
- Nếu User hỏi về thông tin chuyên môn (triệu chứng, DSM-5, rối loạn...), bạn SẼ nhận được thông tin tham khảo trong một tin nhắn System.
- Hãy DỰA VÀO thông tin đó để trả lời.
- Nếu User chỉ trò chuyện, bạn sẽ không nhận được thông tin tham khảo, hãy cứ trò chuyện bình thường.
"""

# --- Memory & Prompt ---
# Bộ nhớ bây giờ sẽ lưu cả system prompt
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)
memory.put(ChatMessage(role="system", content=SYSTEM_PROMPT))

# --- Reranker ---
reranker_model = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# --- Hàm kiểm tra có cần RAG không ---
def should_use_rag(message: str) -> bool:
    keywords = [
        "DSM‑5", "triệu chứng", "rối loạn", "chẩn đoán", "phân loại",
        "trầm cảm", "chán nản", "mất hứng", "vô vụ", "suicidal",
        "lo âu", "hồi hộp", "panic", "stress", "sợ hãi",
        "mất ngủ", "ngủ không ngon", "hypersomnia", "insomnia",
        "thay đổi cân nặng", "mệt mỏi", "thiếu năng lượng", "thiếu tập trung",
        "disorder", "diagnosis", "symptom", "mental disorder"
    ]
    return any(k.lower() in message.lower() for k in keywords)

# --- Hàm RAG retrieval + reranker + tóm tắt ---
def rag_retrieve_and_rerank(message: str, top_k=10, final_top=5) -> str:
    print(f"🔍 [RAG] Truy vấn: '{message}'")
    retriever = vector_index.as_retriever(similarity_top_k=top_k)
    retrieved_docs = retriever.retrieve(message)
    
    if not retrieved_docs:
        print("⚠️ Không tìm thấy tài liệu liên quan.")
        return ""

    doc_texts = [doc.get_content() for doc in retrieved_docs]
    pairs = [(message, text) for text in doc_texts]

    try:
        scores = reranker_model.predict(pairs)
        ranked_docs = [doc for _, doc in sorted(zip(scores, retrieved_docs), key=lambda x: x[0], reverse=True)]
    except Exception as e:
        print(f"❌ Rerank lỗi: {e}")
        ranked_docs = retrieved_docs

    top_docs = ranked_docs[:final_top]
    context_text = "\n\n---\n\n".join([doc.get_content() for doc in top_docs])

    if len(context_text.split()) > 700:
        print("✂️ Context quá dài – đang tóm tắt...")
        try:
            summary_prompt = f"Tóm tắt nội dung tài liệu DSM-5 sau để trả lời câu hỏi: '{message}':\n\n{context_text}"
            context_text = llm.complete(summary_prompt).text # .complete() OK khi chỉ dùng cho tóm tắt
        except Exception as e:
            print(f"❌ Lỗi tóm tắt: {e}")
            context_text = " ".join(context_text.split()[:700])
    
    # Trả về context đã được chuẩn bị
    return f"--- Thông tin tham khảo từ DSM-5 ---\n{context_text}\n--- Hết thông tin tham khảo ---"


# --- SỬA LỖI 2: HÀM CHAT (Dùng .chat() để fix lỗi lặp) ---
def chat_conversation():
    print("\n🤖 Chatbot DSM-5 đã sẵn sàng. Gõ 'quit' để kết thúc.\n")

    while True:
        try:
            user_message = input("Bạn: ").strip()
            if user_message.lower() == "quit":
                print("Chatbot: Cảm ơn bạn đã trò chuyện. Hẹn gặp lại 🌷")
                break
            
            # --- TẠO TIN NHẮN MỚI ---
            new_user_message = ChatMessage(role="user", content=user_message)
            
            # Chuẩn bị 1 list tin nhắn để gửi cho LLM
            messages_to_send = memory.get_all() # Lấy toàn bộ lịch sử (bao gồm cả System Prompt)
            
            # --- BƯỚC 1: LẤY CONTEXT (RAG) ---
            if should_use_rag(user_message):
                context = rag_retrieve_and_rerank(user_message)
                # Thêm context vào list tin nhắn như 1 tin nhắn system
                # (chỉ cho lượt này, không lưu vào memory chính)
                messages_to_send.append(ChatMessage(role="system", content=context))

            # Thêm tin nhắn cuối cùng của người dùng
            messages_to_send.append(new_user_message)

            # --- BƯỚC 2: LLM TRẢ LỜI (FIXED: DÙNG .chat() ) ---
            # .chat() sẽ tôn trọng các stopping_ids (như <|eot_id|>)
            # mà bạn đã setup trong llm.
            response = llm.chat(messages_to_send)
            
            # Lấy nội dung tin nhắn trả lời
            response_text = response.message.content.strip()
            if not response_text:
                response_text = "Mình chưa chắc chắn về điều này, bạn có thể hỏi rõ hơn không?"

            # --- BƯỚC 3: LƯU VÀO BỘ NHỚ ---
            # Chỉ lưu tin nhắn GỐC của user
            memory.put(new_user_message) 
            # Và tin nhắn trả lời của bot
            memory.put(ChatMessage(role="assistant", content=response_text))

            # Hiển thị
            print(f"Chatbot: {response_text}\n")

        except Exception as e:
            print(f"❌ Lỗi không mong muốn: {e}")
            traceback.print_exc() # In chi tiết lỗi



: 